## individual-profile scrapper 
scraping linkedIn individual profile from html file 

In [1]:
import requests, time, random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.support.ui import WebDriverWait
import pyautogui
import keyboard
import glob
import os 
import pandas as pd


In [21]:
## oepn browser 
browser = webdriver.Chrome('/Users/jarvis/Desktop/winter 22/CS for SS2/fianl project/final-project-ocean-s-4/chromedriver')
browser.get('http://www.linkedin.com/uas/login')

<ipython-input-21-c7121a012f55>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('/Users/jarvis/Desktop/winter 22/CS for SS2/fianl project/final-project-ocean-s-4/chromedriver')


In [3]:
## open file and get individual profile link
profile_link = []
for filepath in glob.glob(os.path.join('/Users/jarvis/Desktop/winter 22/CS for SS2/fianl project/final-project-ocean-s-4/web_page/University of California-Davis', '*.html')):
    with open(filepath) as fp:
        soup_test = BeautifulSoup(fp, 'html.parser')
        ## get profile link of the whole page 
        soup_test.find_all('dt', class_='result-lockup__name')
        for links in soup_test.find_all('dt', class_='result-lockup__name'):
            alst = links.find_all('a')
            profile_link.extend([e['href'] for e in alst])
       

### set school name here: 

In [22]:
search_school_name = "University of California-Davis"

In [ ]:
lst_record = []
for link in profile_link[200:300]:
    result = []
    individual_record = {}
    time.sleep(random.uniform(1,2))
    browser.get(link)
    src = browser.page_source
    soup = BeautifulSoup(src, 'lxml')
    top_card = soup.find('div', {'class': 'profile-topcard-content-container mr2'})
    try:
        text = top_card.get_text()
        text = text.split('\n')
    except AttributeError:
        pass 
    for t in text:
        if t.strip():
            result.append(t.strip())
    for i in range(len(result)):
        if "connections" in result[i]:
            try:
                individual_record["current_location"] = result[i-1]
            except IndexError:
                pass
        if "Current" in result[i]:
            try:
                individual_record["current_work"] = result[i+1]
            except IndexError:
                pass
        if "Previous" in result[i]:
            try:
                individual_record["previous_work"] = result[i+1]
            except IndexError:
                pass
        if "Education" in result[i]:
            try:
                individual_record["latest_Education"] = result[i+1]
            except IndexError:
                pass 
        if "NAME_SEARCH" in link:
            degree = soup.find('span',{'data-test-education':"field"})
            try:
                degree = degree.getText().strip()
                individual_record["majro"] = degree
            except AttributeError:
                 pass 
    if individual_record not in lst_record:
        lst_record.append(individual_record)
        

In [9]:
## write out csv file 
df = pd.DataFrame(lst_record)
df['search_school'] = search_school_name
filename = str(search_school_name)+".csv"
df.to_csv(filename,index = True)